In [1]:
import pandas as pd
import os

In [3]:
def read_crop_data(csv_paths, crop):
    dfs = []
    for path in csv_paths:
        df = pd.read_csv(path)
        # Select columns
        df = df[["NAME", "DATE", "PRCP", "TAVG", "TMAX", "TMIN"]]

        # Extract State Abbreviation
        df["STATE"] = df["NAME"].str.extract(r",\s*([A-Z]{2})\s")[0]
        df = df.drop(columns=["NAME"])

        # Extract Year and Month
        df["YEAR"] = df["DATE"].str.split("-").str[0]
        df["MONTH"] = df["DATE"].str.split("-").str[1]
        df = df.drop(columns={"DATE"})

        # Map of numerical month value to month
        month_mapping = {
            "01": "January",
            "02": "February",
            "03": "March",
            "04": "April",
            "05": "May",
            "06": "June",
            "07": "July",
            "08": "August",
            "09": "September",
            "10": "October",
            "11": "November",
            "12": "December",
        }

        # Replace numerical month value with month
        df["MONTH"] = df["MONTH"].replace(month_mapping)

        # Create column for corresponding crop type
        df["CROP"] = crop

        dfs.append(df)

    #Combining dataframes of the same crop
    df_combined = pd.concat(dfs, ignore_index = True)

    return df_combined


In [4]:
# Relative paths for .csv data for each crop
corn_paths = ["noaa_csv_files/Corn_1.csv", "noaa_csv_files/Corn_2.csv"]
soybean_paths = ["noaa_csv_files/Soybean_1.csv", "noaa_csv_files/Soybean_2.csv"]
barley_paths = ["noaa_csv_files/Barley_1.csv", "noaa_csv_files/Barley_2.csv", "noaa_csv_files/Barley_3.csv"]
oats_paths = ["noaa_csv_files/Oats_1.csv", "noaa_csv_files/Oats_2.csv"]

# Crop types
crops = ["corn", "soybean", "barley", "oats"]

#List of .csv paths
paths = [corn_paths, soybean_paths, barley_paths, oats_paths]

#Use for loop and read_crop_data() fo create crop dataframes
crop_data = {}
for path, crop in zip(paths, crops):
    crop_data[crop] = read_crop_data(path, crop)

for crop in crops:
    print(crop_data[crop].head())

for crop in crops:
    print(crop_data[crop][crop_data[crop].isnull().any(axis=1)])

for crop in crops:
    crop_data[crop].to_csv(os.path.join("cleaned_csvs", crop + ".csv"))

   PRCP  TAVG  TMAX  TMIN STATE  YEAR     MONTH  CROP
0  0.09  18.4  28.2   8.5    IA  2013   January  corn
1  0.07  21.4  30.6  12.2    IA  2013  February  corn
2  1.98  26.2  34.4  18.0    IA  2013     March  corn
3  6.16  41.3  51.2  31.3    IA  2013     April  corn
4  7.13  57.0  67.2  46.9    IA  2013       May  corn
   PRCP  TAVG  TMAX  TMIN STATE  YEAR      MONTH     CROP
0  2.42   NaN   NaN   NaN    NE  2020  September  soybean
1  0.75   NaN   NaN   NaN    NE  2020    October  soybean
2  1.27   NaN   NaN   NaN    NE  2020   November  soybean
3  8.20   NaN   NaN   NaN    NE  2021     August  soybean
4  0.40   NaN   NaN   NaN    NE  2021   November  soybean
   PRCP  TAVG  TMAX  TMIN STATE  YEAR     MONTH    CROP
0  1.04  16.5  26.9   6.1    ID  2013   January  barley
1  0.69  22.9  31.9  14.0    ID  2013  February  barley
2  1.54  32.1  42.5  21.7    ID  2013     March  barley
3  1.52  38.2  49.5  27.0    ID  2013     April  barley
4  1.25  49.9  63.2  36.6    ID  2013       May 

In [14]:
soybean_df = crop_data["soybean"]
growing_months = ['June', 'July', 'August', 'September']

soybean_df = soybean_df[(soybean_df['STATE']=='IA')]
soybean_df = soybean_df[soybean_df['MONTH'].isin(growing_months)]
grouped_df = soybean_df.groupby('YEAR').agg({'PRCP': 'mean', 'TMIN': 'mean', 'TMAX': 'mean', 'TAVG': 'mean'}).reset_index()

grouped_df


,YEAR,PRCP,TMIN,TMAX,TAVG
0,2013,2.628333,59.175,81.050,70.100
1,2014,5.307333,57.600,78.250,67.925
2,2015,5.379167,58.925,80.575,69.750
3,2016,3.317857,59.000,82.400,70.700
4,2017,3.068333,57.150,82.025,69.575
5,2018,5.533750,59.775,81.275,70.575
6,2019,3.856429,58.950,81.125,70.050
7,2020,1.890000,59.325,83.850,71.600
8,2021,2.830769,59.375,84.775,72.100
9,2022,1.980000,58.525,84.850,71.700
